In [1]:
import numpy as np
import json

from microfaune import audio, plot

# Labeling with Audacity

In [4]:
def convert_label_audacity_csv(file_path):

    file = open(file_path, 'r')
    lines_list = file.read()
    lines_list = lines_list.split('\n')
    lines_list = lines_list[::2]
    lines_list = lines_list[:-1]
    
    start_points = [round(float(line.split('\t')[0]), 2) for line in lines_list]
    end_points = [round(float(line.split('\t')[1]), 2) for line in lines_list]
    
    tab = np.array((start_points, end_points)).transpose()
    np.savetxt(file_path[:-4]+'.csv', tab, delimiter=";")
    
    return None


def convert_label_audacity_json(csv_file):
    
    data = np.genfromtxt(csv_file, delimiter=";")
    shape_data = np.shape(data)
    start_time = data[:,0]
    end_time = data[:,1]
    
    data_dict = []
    
    for i in range(shape_data[0]):
        data_dict.append({
            'id': 'wavesurfer_' + str(i), 
            'start': start_time[i], 
            'end': end_time[i],
            'annotation': 'bird'
        })
    
    with open(csv_file[:-4]+'.json', 'w') as outfile:
        json.dump(data_dict, outfile)
    
    return None

In [5]:
txt_file = "C:/Users/Camille/Documents/MicroFaune/Git/labeling/test_audacity/test_labeling.txt"
convert_label_audacity_csv(txt_file)

csv_file = txt_file[:-4]+'.csv'
convert_label_audacity_json(csv_file)

# Labeling with Louis audio annotator

In [59]:
def number_bird_song(json_file_path):
    
    with open(json_file_path) as json_data:
        data_dict = json.load(json_data)
    
    return len(data_dict)


def prop_bird_song(json_file_path, wav_file_path):
    
    fs, data = audio.load_wav(wav_file_path)
    total_duration = len(data)/fs
    
    with open(json_file_path) as json_data:
        data_dict = json.load(json_data)
        
    bird_song_duration = 0
    
    for label in data_dict:
        bird_song_duration += label['end'] - label['start']
    
    return round(bird_song_duration/total_duration, 4)


def audio_charac_function_audio(json_file_path, wav_file_path):
    
    fs, data = audio.load_wav(wav_file_path)
    charac_func = np.zeros((len(data),1))
    
    with open(json_file_path) as json_data:
        data_dict = json.load(json_data)
        
    for label in data_dict:
        indx_start = int(label['start']*fs)
        indx_end = int(label['end']*fs)
        charac_func[indx_start:indx_end+1,0] = 1
    
    return charac_func

def audio_charac_function_spec(window_length, overlap, charac_func_audio):
    
    size_spec = 2+int(duration/(window_length*(1-overlap)))
    size_audio = len(charac_func_audio)
    regroup_factor = int(size_audio / size_spec)
    
    charac_func_spec = np.zeros((size_spec,1))
    
    for i in range(size_spec):
        label_local = np.mean(charac_func_audio[i*regroup_factor : (i+1)*regroup_factor])
        if label_local>0.5:
            charac_func_spec[i] = 1
    
    return charac_func_spec

In [73]:
json_file_path = "C:/Users/Camille/Documents/MicroFaune/audio-annotator/static/json/test_labeling_labeled.json"
wav_file_path = "C:/Users/Camille/Documents/MicroFaune/audio-annotator/static/wav/test_labeling.wav"

print("Number of bird songs labelised: " + str(number_bird_song(json_file_path)) )
print("Proportion of bird songs during extract: " + str(prop_bird_song(json_file_path, wav_file_path)*100) + " %" )

window_length = 0.020
overlap = 0.55

charac_func_audio = audio_charac_function_audio(json_file_path, wav_file_path)
charac_func_spec = audio_charac_function_spec(window_length, overlap, charac_func_audio)

Number of bird songs labelised: 26
Proportion of bird songs during extract: 56.74 %


In [74]:
#check length
[spec, t, f, x_fs] = audio.file2spec(wav_file_path, scale_spec="linear", window_length=window_length, overlap=overlap)
print(len(t))
print(len(charac_func_spec))

13485
13485


In [75]:
#check proportion
yes_audio=np.where(charac_func_audio>0)
print(len(yes_audio[0])/len(charac_func_audio))
yes_spec=np.where(charac_func_spec>0)
print(len(yes_spec[0])/len(charac_func_spec))

0.5674447813701579
0.5667037449017427
